In [1]:
from glob import glob
import re

import pandas as pd

In [2]:
data = []
AGS_RE = re.compile('\d{3}')
PATHS = 'raw/Pflegestatistik_2013_Thüringen_{}.xls'

In [3]:
file_mapping = {
    'beds': '1',
    'recipients': '2',
    'personal': '3',
    'costs': '4'
}

In [4]:
def get_values(filename, mapping, column_rename=('key', 'value'), prefix='personal', indicator='key', skip_on=None, skiprows=None):
    ex = pd.ExcelFile(filename)
    cache = set()
    for sheet in ex.sheet_names:
        match = AGS_RE.search(sheet)
        if match is None:
            continue
        ags = '16' + match.group(0)
        df = ex.parse(sheet, skiprows=skiprows)
        df = df.rename(columns={c: v for c, v in zip(df.columns, column_rename)})
        for i, row in df.iterrows():
            if pd.isnull(row['key']):
                continue
            if skip_on is not None and row['key'] in skip_on:
                ags = None
                continue
            for k, v in mapping.items():
                if k in row['key']:
                    if isinstance(row['value'], str):
                        val = row['value'].replace('-', '0').strip()
                        if val == '.':
                            val = None
                        else:
                            val = float(val)
                    else:
                        val = row['value']
                    cache_key = '%s-%s' % (ags, v)
                    if cache_key not in cache:
                        yield ags, prefix, v, val
                        cache.add(cache_key)


In [5]:
filename = PATHS.format(file_mapping['beds'])

bed_mapping = {
  '1-Bett-Zimmern': '1-bed',
  '2-Bett-Zimmern': '2-bed',
  '3-Bett-Zimmern': '3-bed',
  '4 und mehr-Bett-Zimmern': '4-bed'
}


l = list(get_values(filename, bed_mapping, prefix='beds', skiprows=12))
print(l)
data.extend(l)

[('16051', 'beds', '1-bed', 1540), ('16051', 'beds', '2-bed', 965), ('16051', 'beds', '3-bed', 0.0), ('16051', 'beds', '4-bed', 0.0), ('16052', 'beds', '1-bed', 1043), ('16052', 'beds', '2-bed', 218), ('16052', 'beds', '3-bed', 0.0), ('16052', 'beds', '4-bed', 0.0), ('16053', 'beds', '1-bed', 668), ('16053', 'beds', '2-bed', 438), ('16053', 'beds', '3-bed', 0.0), ('16053', 'beds', '4-bed', 0.0), ('16054', 'beds', '1-bed', 280), ('16054', 'beds', '2-bed', 118), ('16054', 'beds', '3-bed', 0.0), ('16054', 'beds', '4-bed', 0.0), ('16055', 'beds', '1-bed', 623), ('16055', 'beds', '2-bed', 248), ('16055', 'beds', '3-bed', 0.0), ('16055', 'beds', '4-bed', 0.0), ('16056', 'beds', '1-bed', 391), ('16056', 'beds', '2-bed', 172), ('16056', 'beds', '3-bed', 0.0), ('16056', 'beds', '4-bed', 0.0), ('16061', 'beds', '1-bed', 683.0), ('16061', 'beds', '2-bed', 235.0), ('16061', 'beds', '3-bed', 29.0), ('16061', 'beds', '4-bed', 12.0), ('16062', 'beds', '1-bed', 913), ('16062', 'beds', '2-bed', 460), (

In [6]:
filename = PATHS.format(file_mapping['recipients'])

recipient_mapping = {
    'Pflegestufe I ': 'recipients_nursing_class_1',
    'Pflegestufe II ': 'recipients_nursing_class_2',
    'Pflegestufe III ': 'recipients_nursing_class_3',
    'Pflegestufe zugeordnet': 'recipients_nursing_class_unknown',
}


l = list(get_values(filename, recipient_mapping, column_rename=('key', '_blank_', 'value'), prefix='recipients', skiprows=12))
print(l)
data.extend(l)

[('16051', 'recipients', 'recipients_nursing_class_1', 1019), ('16051', 'recipients', 'recipients_nursing_class_2', 826), ('16051', 'recipients', 'recipients_nursing_class_3', 456), ('16051', 'recipients', 'recipients_nursing_class_unknown', 20), ('16052', 'recipients', 'recipients_nursing_class_1', 367), ('16052', 'recipients', 'recipients_nursing_class_2', 556), ('16052', 'recipients', 'recipients_nursing_class_3', 318), ('16052', 'recipients', 'recipients_nursing_class_unknown', 4), ('16053', 'recipients', 'recipients_nursing_class_1', 399), ('16053', 'recipients', 'recipients_nursing_class_2', 475), ('16053', 'recipients', 'recipients_nursing_class_3', 209), ('16053', 'recipients', 'recipients_nursing_class_unknown', 2), ('16054', 'recipients', 'recipients_nursing_class_1', 100), ('16054', 'recipients', 'recipients_nursing_class_2', 157), ('16054', 'recipients', 'recipients_nursing_class_3', 113), ('16054', 'recipients', 'recipients_nursing_class_unknown', 0.0), ('16055', 'recipien

In [7]:
personal_mapping = {
    'Vollzeitbeschäftigt': 'fully_employed',
    'Teilzeitbeschäftigt': 'part-time',
    'Pflege und Betreuung': 'nursing_and_care',
    'soziale Betreuung': 'social_care',
    'zusätzliche Betreuung (§ 87b SGB XI)': 'other_care',
    'Personal insgesamt': 'total_personnel'
}


filename = PATHS.format(file_mapping['personal'])

l = list(get_values(filename, personal_mapping, prefix='personal', skiprows=10))
print(l)
data.extend(l)

[('16051', 'personal', 'total_personnel', 1579.0), ('16051', 'personal', 'nursing_and_care', 1142.0), ('16051', 'personal', 'social_care', 91.0), ('16051', 'personal', 'other_care', 80.0), ('16051', 'personal', 'fully_employed', 486.0), ('16051', 'personal', 'part-time', 903.0), ('16052', 'personal', 'total_personnel', 849.0), ('16052', 'personal', 'nursing_and_care', 661.0), ('16052', 'personal', 'social_care', 38.0), ('16052', 'personal', 'other_care', 39.0), ('16052', 'personal', 'fully_employed', 298.0), ('16052', 'personal', 'part-time', 478.0), ('16053', 'personal', 'total_personnel', 754.0), ('16053', 'personal', 'nursing_and_care', 558.0), ('16053', 'personal', 'social_care', 45.0), ('16053', 'personal', 'other_care', 31.0), ('16053', 'personal', 'fully_employed', 152.0), ('16053', 'personal', 'part-time', 501.0), ('16054', 'personal', 'total_personnel', 310.0), ('16054', 'personal', 'nursing_and_care', 233.0), ('16054', 'personal', 'social_care', 13.0), ('16054', 'personal', '

In [8]:
cost_mapping = {
    'Pflegeklasse 1': 'costs_nursing_class_1',
    'Pflegeklasse 2': 'costs_nursing_class_2',
    'Pflegeklasse 3': 'costs_nursing_class_3',
    'Verpflegung': 'food',
}

filename = PATHS.format(file_mapping['costs'])

l = list(get_values(filename, cost_mapping, prefix='costs', skiprows=0))
print(l)
data.extend(l)

[('16051', 'costs', 'costs_nursing_class_1', 37.05), ('16051', 'costs', 'costs_nursing_class_2', 50.74), ('16051', 'costs', 'costs_nursing_class_3', 67.47), ('16051', 'costs', 'food', 20.44), ('16052', 'costs', 'costs_nursing_class_1', 34.27), ('16052', 'costs', 'costs_nursing_class_2', 46.87), ('16052', 'costs', 'costs_nursing_class_3', 61.48), ('16052', 'costs', 'food', 18.91), ('16053', 'costs', 'costs_nursing_class_1', 40.23), ('16053', 'costs', 'costs_nursing_class_2', 53.54), ('16053', 'costs', 'costs_nursing_class_3', 70.04), ('16053', 'costs', 'food', 20.51), ('16054', 'costs', 'costs_nursing_class_1', 40.54), ('16054', 'costs', 'costs_nursing_class_2', 52.63), ('16054', 'costs', 'costs_nursing_class_3', 67.7), ('16054', 'costs', 'food', 19.69), ('16055', 'costs', 'costs_nursing_class_1', 35.26), ('16055', 'costs', 'costs_nursing_class_2', 49.05), ('16055', 'costs', 'costs_nursing_class_3', 65.61), ('16055', 'costs', 'food', 20.32), ('16056', 'costs', 'costs_nursing_class_1', 3

In [9]:
df = pd.DataFrame(data)
df = df.rename(columns={
    0: 'ags',
    1: 'type',
    2: 'key',
    3: 'value'
})
df['state'] = 'Thüringen'
df.to_csv('csvs/thueringen.csv', index=False)
df.head()

,ags,type,key,value,state
0,16051,beds,1-bed,1540.0,Thüringen
1,16051,beds,2-bed,965.0,Thüringen
2,16051,beds,3-bed,0.0,Thüringen
3,16051,beds,4-bed,0.0,Thüringen
4,16052,beds,1-bed,1043.0,Thüringen


In [10]:
df[df['ags'] == '16051']

,ags,type,key,value,state
0,16051,beds,1-bed,1540.00,Thüringen
1,16051,beds,2-bed,965.00,Thüringen
2,16051,beds,3-bed,0.00,Thüringen
3,16051,beds,4-bed,0.00,Thüringen
92,16051,recipients,recipients_nursing_class_1,1019.00,Thüringen
93,16051,recipients,recipients_nursing_class_2,826.00,Thüringen
94,16051,recipients,recipients_nursing_class_3,456.00,Thüringen
95,16051,recipients,recipients_nursing_class_unknown,20.00,Thüringen
184,16051,personal,total_personnel,1579.00,Thüringen
185,16051,personal,nursing_and_care,1142.00,Thüringen
